# Setup simulations based on the process I used for SAMPLing

In [2]:
import glob as glob
import os as os
import subprocess as sp
import numpy as np

import parmed as pmd
from paprika.align import zalign, rotate_about_z, get_theta
from paprika.dummy import add_dummy, write_dummy_mol2, write_dummy_frcmod
from paprika.tleap import System

Niel ran...

- 1 microsecond on endpoints
- Minimum 5 ns, maximum 50 ns on interior windows, with convergence value for attach and release 0.02, and 0.1 for pull (with TI)

First, I'd like to clean the `confirm-original` and `smirnoff` directories of output files because I can't remember what I was doing before we switched to SAMPLing. I *think* I recall saying that I ran simulations quickly, but accidentally omitted the equilibration phase. This will be a good opportunity to do it consistently, and log the process.

In [10]:
files = ['minimize.in',
        'minimize.mdinfo',
        'minimize.out',
        'minimize.rst7',
        'traj.in',
        'traj.mden',
        'traj.nc',
        'traj.out',
        'traj.rst7']

directories = glob.glob('systems/a-bam-p/confirm-original/*') + \
              glob.glob('systems/a-bam-p/smirnoff/*')

In [12]:
for directory in directories:
    for file in files:
        try:
            os.remove(os.path.join(directory, file))
        except:
            pass

I already have a restraint file written in each directory, which was written by the `02...` notebooks. To run analysis during the simulation, I'll have to import those here.

However, while waiting to see if the TI-block convergence values (from the standard deviation of the window-to-window bootstrapped integrations) is the same as the `scale_w` method, I'll just minimize, equilibrate, and run 5 ns in each window.

## Confirm-original
The original topology uses HMR with `dt = 0.004`.


In [34]:
!cat 'systems/a-bam-p/confirm-original/simulate.py'

import os as os
import shutil as shutil
import re as re

import paprika as paprika
from paprika.amber import Simulation

import logging

logging.basicConfig(
    filename='tscc.log',
    format='%(asctime)s %(message)s',
    datefmt='%Y-%m-%d %I:%M:%S %p',
    level=logging.DEBUG)
logging.info('Started logging...')
logging.info(paprika.__version__)

ntwprt = 146
nstlim = 1250000
dt = 0.004
initial_topology = 'full.hmr.topo'
initial_coordinates = 'full.crds'

sim = Simulation()

# Minimization
sim.executable = 'pmemd.cuda'
sim.topology = initial_topology
sim.prefix = 'minimize'
sim.inpcrd = initial_coordinates
sim.path = './'
sim.ref = initial_coordinates
sim.config_pbc_min()
sim.cntrl['maxcyc'] = 50000
sim.cntrl['ncyc'] = 5000
sim.cntrl['ntr'] = 1
sim.cntrl['restraint_wt'] = 50.0
sim.cntrl['restraintmask'] = "'@DUM | :10@C4 | :10@N1'"
sim.cntrl['cut'] = 9.0
sim.restraint_file = 'disang.rest'
sim.run(fail_ok=False)

# Equilibration
sim.config_pbc_md()
sim.executable = 'pmemd.cuda'
sim.t

In [31]:
directories = glob.glob('systems/a-bam-p/confirm-original/*')
directories = [i for i in directories if os.path.isdir(i)]
for directory in directories:
    try:
        os.symlink('../simulate.py', os.path.join(directory, 'simulate.py'))
    except:
        os.remove(os.path.join(directory, 'simulate.py'))
        os.symlink('../simulate.py', os.path.join(directory, 'simulate.py'))
    try:
        os.symlink('../tscc-driver.sh', os.path.join(directory, 'tscc-driver.sh'))
    except:
        os.remove(os.path.join(directory, 'tscc-driver.sh'))
        os.symlink('../tscc-driver.sh', os.path.join(directory, 'tscc-driver.sh'))

## SMIRNOFF
This topology does not use HMR and sets `dt = 0.002`.

In [32]:
!cat 'systems/a-bam-p/smirnoff/simulate.py'

import os as os
import shutil as shutil
import re as re

import paprika as paprika
from paprika.amber import Simulation

import logging

logging.basicConfig(
    filename='tscc.log',
    format='%(asctime)s %(message)s',
    datefmt='%Y-%m-%d %I:%M:%S %p',
    level=logging.DEBUG)
logging.info('Started logging...')
logging.info(paprika.__version__)

ntwprt = 146
nstlim = 2500000
dt = 0.002
initial_topology = 'smirnoff.prmtop'
initial_coordinates = 'smirnoff.inpcrd'

sim = Simulation()

# Minimization
sim.executable = 'pmemd.cuda'
sim.topology = initial_topology
sim.prefix = 'minimize'
sim.inpcrd = initial_coordinates
sim.path = './'
sim.ref = initial_coordinates
sim.config_pbc_min()
sim.cntrl['maxcyc'] = 50000
sim.cntrl['ncyc'] = 5000
sim.cntrl['ntr'] = 1
sim.cntrl['restraint_wt'] = 50.0
sim.cntrl['restraintmask'] = "'@DUM | :10@C4 | :10@N1'"
sim.cntrl['cut'] = 9.0
sim.restraint_file = 'disang.rest'
sim.run(fail_ok=False)

# Equilibration
sim.config_pbc_md()
sim.executable = 'pmemd.cud

In [33]:
directories = glob.glob('systems/a-bam-p/smirnoff/*')
directories = [i for i in directories if os.path.isdir(i)]
for directory in directories:
    try:
        os.symlink('../simulate.py', os.path.join(directory, 'simulate.py'))
    except:
        os.remove(os.path.join(directory, 'simulate.py'))
        os.symlink('../simulate.py', os.path.join(directory, 'simulate.py'))
    try:
        os.symlink('../tscc-driver.sh', os.path.join(directory, 'tscc-driver.sh'))
    except:
        os.remove(os.path.join(directory, 'tscc-driver.sh'))
        os.symlink('../tscc-driver.sh', os.path.join(directory, 'tscc-driver.sh'))